In [ ]:
import torch
from torchvision import transforms, models
import numpy as np
import os
from PIL import Image
import os 
import pickle

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# load the model (dinov2->vector size 368)
model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14').to(device)


Using device: cpu


Using cache found in /home/jsaavedr/.cache/torch/hub/facebookresearch_dinov2_main
/home/jsaavedr/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/home/jsaavedr/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/home/jsaavedr/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


In [3]:
# Definimos las transformaciones para la imagen
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]),
    ])

def compute_embeddings(lfiles) :
    n = len(lfiles)
    dim = 384
    features = np.zeros((n, dim), dtype = np.float32)
    with torch.no_grad():
        for i, ifile in enumerate(lfiles) :
            image = Image.open(ifile).convert('RGB')
            image = preprocess(image).unsqueeze(0).to(device)
            features[i,:] = model(image).cpu()[0,:]
            if i%10 == 0 :
                print('{}/{}'.format(i, n))
    return features

In [ ]:
data_dir = '/home/data/Homy'
fdata = '/home/data/Homy/eval.txt'
lfiles = []
with open(fdata) as f:
    for ifile in f :
        ifile = os.path.join(data_dir, ifile.strip())
        lfiles.append(ifile)
features = compute_embeddings(lfiles)
print(features.shape)
with open('data/dinov2_homy.pk', 'wb') as f:
    pickle.dump(lfiles, f)
np.save("data/dinov2_homy_feats.npy", features)
print('saving data ok')                  